In [1]:
# In this we would learn about the concept of broadcast variables

In [2]:
# We should broadcase python objects so that it can be sent to every worker node once and store it there for 
# efficiency. Worker node can use that python object whenever needed (no need to fetch it from master)

# if you don't use broadcast() then master node would end up sending object to worker nodes multiple times and it 
# would be inefficient if object is so huge.

In [3]:
'''
Broadcast objects to the executors, such that they are always there whenever needed

Just use sc.broadcast() to ship off whatever you want

Then Use .value to get the shipped object back on executors  

'''

'\nBroadcast objects to the executors, such that they are always there whenever needed\n\nJust use sc.broadcast() to ship off whatever you want\n\nThen Use .value() to get the shipped object back on executors  \n\n'

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark import SparkConf,SparkContext

In [8]:
conf = SparkConf().setMaster("local").setAppName("PopularMovieNicer")
sc = SparkContext(conf=conf)

In [65]:
def get_moviename_dict():
    moviename = dict()
    # opening file in binary format since it is having one byte character i.e é
    with open('/Users/hdagar3/Documents/Spark_Things/Spark_Course_Files/ml-100k/u.item','rb') as f:
        for line in f:   # line is chunk of bytes now
            info = line.split(bytes('|',encoding="utf-8"))  # to split bytes, we need byte character
            moviename[int(info[0])] = info[1]
    return moviename

# dictionary = get_moviename_dict()

# for key,value in dictionary.items():
#    print(key,value)

In [57]:
'<sdfé>'.encode("utf-8").decode()  # encoding on string but decoding on byte

'<sdfé>'

In [69]:
'<678|dfé>'.split('|')    # <string>.split(<string>)  --> right
                          # <bytes>.split(<bytes>)    --> right
                          # <string>.split(<bytes>)   --> wrong
                          # <byte>.split(<string>)    --> wrong

['<678', 'dfé>']

In [68]:
'678|dfé'.encode("utf-8").split(bytes('|',encoding="utf-8"))

[b'678', b'df\xc3\xa9']

In [21]:
base_rdd = sc.textFile('file:///Users/hdagar3/Documents/Spark_Things/Spark_Course_Files/ml-100k/u.data')

In [72]:
dictFetcher = sc.broadcast(get_moviename_dict())  # it will broadcast this to all worker nodes in a cluster
# dictionary can be fetched from this dictFetcher using .value

In [74]:
movie_one_rdd = base_rdd.map(lambda line : (int(line.split()[1]),1))

In [75]:
movie_noofwatches_rdd = movie_one_rdd.reduceByKey(lambda x,y:x+y)

In [78]:
noofwatches_movie_rdd = movie_noofwatches_rdd.map(lambda x:(x[1],x[0]))

In [84]:
sorted_rdd = noofwatches_movie_rdd.sortByKey(ascending=False)

In [85]:
results = sorted_rdd.collect()

In [86]:
my_original_dict = dictFetcher.value # it is giving us a dictionary

In [88]:
for result in results:
    print(result[0],my_original_dict[result[1]])

In [ ]:
# END